# Create Embeddings

In [13]:
from transformers import AutoTokenizer, AutoModel
import torch
import csv
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import csv
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report
import ast


In [14]:
# Code Bert Tokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base",force_download=False)
model = AutoModel.from_pretrained("microsoft/codebert-base",force_download=False)

In [15]:
# Initialize data object with ground truths
applicationGtPath = "ground_truths/v_imen/pos/posapplication.txt"
utilityGtPath = "ground_truths/v_imen/pos/posutility.txt"
entityGtPath = "ground_truths/v_imen/pos/posentity.txt"

entity=[]
application=[]
utility=[]
data={}
inp = open(applicationGtPath, "r")
application = inp.read().splitlines()
inp.close()
for c in application:
    data[c]=["application"]
inp = open(utilityGtPath, "r")
utility = inp.read().splitlines()
inp.close()
for c in utility:
    data[c]=["utility"]
inp = open(entityGtPath, "r")
entity = inp.read().splitlines()
inp.close()
for c in entity:
    data[c]=["entity"]

print(data)

{'com.rafsan.inventory.MainApp': ['application'], 'com.rafsan.inventory.controller.admin.AdminController': ['application'], 'com.rafsan.inventory.interfaces.CategoryInterface': ['application'], 'com.rafsan.inventory.controller.category.AddController': ['application'], 'com.rafsan.inventory.controller.category.CategoryController': ['application'], 'com.rafsan.inventory.controller.category.EditController': ['application'], 'com.rafsan.inventory.interfaces.EmployeeInterface': ['application'], 'com.rafsan.inventory.controller.employee.AddController': ['application'], 'com.rafsan.inventory.controller.employee.EditController': ['application'], 'com.rafsan.inventory.controller.employee.EmployeeController': ['application'], 'com.rafsan.inventory.interfaces.ProductInterface': ['application'], 'com.rafsan.inventory.controller.product.AddController': ['application'], 'com.rafsan.inventory.controller.product.EditController': ['application'], 'com.rafsan.inventory.controller.product.ProductControll

In [16]:
# Loop through source code in path specified and add embeddings in data object
path = './src_code/pos/src_code_formatted/'

for elem in data:
    n=0
    allembadding=[]
    with open(path+elem.lower()+'.java',encoding = "ISO-8859-1",errors="ignore") as f:
        print(elem)
        lines=f.read()
        code_tokenss = tokenizer.tokenize(lines)
        while(len(code_tokenss)>(n)*510):
            code_tokens=code_tokenss[(n)*510:(n+1)*510]
            tokens = [tokenizer.cls_token] +  code_tokens + [tokenizer.sep_token]
            tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
            context_embeddings = model(torch.tensor(tokens_ids)[None, :])[0]
            embaddings= context_embeddings[0][0].tolist()
            allembadding.append(embaddings)
            n=n+1
        npmatrix=np.matrix(allembadding)
        meanmatrix=npmatrix.mean(0)
        arr=np.array(meanmatrix[0]).flatten()
        data[elem].append(arr.tolist())


Token indices sequence length is longer than the specified maximum sequence length for this model (801 > 512). Running this sequence through the model will result in indexing errors


com.rafsan.inventory.MainApp
com.rafsan.inventory.controller.admin.AdminController
com.rafsan.inventory.interfaces.CategoryInterface
com.rafsan.inventory.controller.category.AddController
com.rafsan.inventory.controller.category.CategoryController
com.rafsan.inventory.controller.category.EditController
com.rafsan.inventory.interfaces.EmployeeInterface
com.rafsan.inventory.controller.employee.AddController
com.rafsan.inventory.controller.employee.EditController
com.rafsan.inventory.controller.employee.EmployeeController
com.rafsan.inventory.interfaces.ProductInterface
com.rafsan.inventory.controller.product.AddController
com.rafsan.inventory.controller.product.EditController
com.rafsan.inventory.controller.product.ProductController
com.rafsan.inventory.interfaces.PurchaseInterface
com.rafsan.inventory.controller.purchase.AddController
com.rafsan.inventory.controller.purchase.PurchaseController
com.rafsan.inventory.interfaces.ReportInterface
com.rafsan.inventory.controller.report.ReportC

In [17]:
# Write embeddings to csv file
fileName = 'posEmbeddings.csv'
with open(fileName, mode='w') as employee_file:
    employee_writer = csv.writer(employee_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for n in data:
        employee_writer.writerow([n,data[n][0], data[n][1]])

# Class classification (assign Utility, Application, or Entity Tag)

In [18]:
X=[]
y=[]
Xpos=[]
ypos=[]
namespos=[]
names=[]

fileName = 'posEmbeddings.csv'
with open(fileName, newline='') as classEmbaddings:
    reader = csv.reader(classEmbaddings, delimiter=';')
    for row in reader:
        X.append(ast.literal_eval(row[2]))
        y.append(row[1])
        names.append(row[0])

X_train, X_test, y_trainnames, y_testnames = train_test_split(X, names, random_state = 0,test_size=0.7)
y_train=[]
y_test=[]
for yt in y_trainnames:
    y_train.append(y[names.index(yt)])
for yt in y_testnames:
    y_test.append(y[names.index(yt)])
utilityyes=False
for yitem in y_train:
    if(yitem=="utility"):
        utilityyes=True
if not utilityyes:
    for i,yitem in enumerate(y):
        if(yitem=="utility"):
            y_train.append("utility")
            X_train.append(X[i])
            break

## SVM

In [19]:
from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train)
dtree_predictions = dtree_model.predict(X_test)
cm = confusion_matrix(y_test, dtree_predictions)

from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, dtree_predictions)

In [20]:
from sklearn.svm import SVC
svm_model_linear = SVC(kernel = 'linear', C = 2).fit(X_train, y_train)
svm_predictions = svm_model_linear.predict(X_test)
# model accuracy for X_test  
accuracysvm = svm_model_linear.score(X_test, y_test)
  
# creating a confusion matrix
cmsvm = confusion_matrix(y_test, svm_predictions)
print(accuracysvm)
print(cmsvm)
print(classification_report(y_test, svm_predictions, target_names=['Class 1', 'Class 2', 'Class 3']))

0.9230769230769231
[[19  0  0]
 [ 1 16  0]
 [ 2  0  1]]
              precision    recall  f1-score   support

     Class 1       0.86      1.00      0.93        19
     Class 2       1.00      0.94      0.97        17
     Class 3       1.00      0.33      0.50         3

    accuracy                           0.92        39
   macro avg       0.95      0.76      0.80        39
weighted avg       0.93      0.92      0.91        39



In [21]:
# training a KNN classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 5).fit(X_train, y_train)
  
# accuracy on X_test
accuracyknn = knn.score(X_test, y_test)
print (accuracyknn)
  
# creating a confusion matrix
knn_predictions = knn.predict(X_test) 
cmknn = confusion_matrix(y_test, knn_predictions)
print(cmknn)
print(classification_report(y_test, knn_predictions, target_names=['Class 1', 'Class 2', 'Class 3']))

0.6666666666666666
[[19  0  0]
 [10  7  0]
 [ 3  0  0]]
              precision    recall  f1-score   support

     Class 1       0.59      1.00      0.75        19
     Class 2       1.00      0.41      0.58        17
     Class 3       0.00      0.00      0.00         3

    accuracy                           0.67        39
   macro avg       0.53      0.47      0.44        39
weighted avg       0.73      0.67      0.62        39



/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf_predictions = clf.predict(X_test)
cmclf = confusion_matrix(y_test, clf_predictions)
# accuracy on X_test
accuracycmclf = clf.score(X_test, y_test)
print (accuracycmclf)
print(cmclf)
print(classification_report(y_test, clf_predictions, target_names=['Class 1', 'Class 2', 'Class 3']))

0.8717948717948718
[[19  0  0]
 [ 3 14  0]
 [ 2  0  1]]
              precision    recall  f1-score   support

     Class 1       0.79      1.00      0.88        19
     Class 2       1.00      0.82      0.90        17
     Class 3       1.00      0.33      0.50         3

    accuracy                           0.87        39
   macro avg       0.93      0.72      0.76        39
weighted avg       0.90      0.87      0.86        39



/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
# training a Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(X_train, y_train)
gnb_predictions = gnb.predict(X_test)
  
# accuracy on X_test
gnb_accuracy = gnb.score(X_test, y_test)
  
# creating a confusion matrix
gnb_cm = confusion_matrix(y_test, gnb_predictions)
print(gnb_accuracy)
print(gnb_cm)

print(classification_report(y_test, gnb_predictions, target_names=['Class 1', 'Class 2', 'Class 3']))

0.8461538461538461
[[19  0  0]
 [ 4 13  0]
 [ 2  0  1]]
              precision    recall  f1-score   support

     Class 1       0.76      1.00      0.86        19
     Class 2       1.00      0.76      0.87        17
     Class 3       1.00      0.33      0.50         3

    accuracy                           0.85        39
   macro avg       0.92      0.70      0.74        39
weighted avg       0.88      0.85      0.84        39

